# 

In [1]:
import pandas as pd
from bioinfokit.analys import Fasta
import os
# helpers function

def get_fasta(filename):
    n = 0
    sequence = ""
    te_name = ""
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('>'):
                n = n + 1
                if not n == 1:
                    first_n = False
                    yield te_name, sequence
                te_name = line[1:].strip()
                sequence = ""
            else:
                if te_name:
                    sequence = sequence + line.strip()
    yield te_name, sequence

def get_len_fasta(filename):
    counter = {}
    with open(filename) as f:
        te_name = ""
        for line in f.readlines():
            if line.startswith('>'):
                te_name = line[1:].strip()
                counter[te_name] = 0
            else:
                if te_name:
                    counter[te_name] = counter[te_name] + len(line.strip()) - line.count("~")
    return counter

def get_from_te(fasta, te_name):
    te_len = get_len_fasta(fasta)[te_name]
    return int(te_len - te_len * dif_percent / 100)

def get_to_te(fasta, te_name):
    te_len = get_len_fasta(fasta)[te_name]
    return int(te_len + te_len * dif_percent / 100)

def read_fasta(filename):
    return pd.read_csv(filename, sep="\t", header=None).rename(headers_dict, axis=1)

def save_fasta(df, filename):
    df.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=False)

def _create_part_raport(df):
    if len(df) == 0:
        return pd.DataFrame(
            {
                'qseqid': [],
                'sseqid': [],
                'start_te': [],
                'end_te': [],
                'start_genome_te': [],
                'end_genome_te': [],
    
                
                
                'qstart1': [],
                'qend1': [],
                'sstart1': [],
                'send1': [],
                
                'qstart2': [],
                'qend2': [],
                'sstart2': [],
                'send2': [],
    
                'length': [],
                'length_genome': [],
            }
        )
    # there is one qseqid
    # it should be sortet to q_start
    df = df.sort_values(by=['sseqid', 'qstart', 'qend', 'sstart', 'send'])
    start = df[:-1].reset_index(drop=True)
    end = df[1:].reset_index(drop=True)
    return pd.DataFrame(
        {
            'qseqid':df.qseqid.values[0],
            'sseqid':df.sseqid.values[0],
            'start_te': start.qend,
            'end_te': end.qstart,
            'start_genome_te': start.send,
            'end_genome_te': end.sstart,

            
            
            'qstart1': end.qstart,
            'qend1': end.qend,
            'sstart1': end.sstart,
            'send1': end.send,
            
            'qstart2': start.qstart,
            'qend2': start.qend,
            'sstart2': start.sstart,
            'send2': start.send,

            'length': end.qstart - start.qend,
            'length_genome': end.sstart - start.send,
        }
    )


def create_report(df):
    series_list = []
    for qseqid in df.qseqid.unique():
        okno = df[df.qseqid == qseqid]
        for sseqid in okno.sseqid.unique():
            okno2 = okno[(okno.sseqid == sseqid)]
            series_list.append(_create_part_raport(okno2))
    return pd.concat(series_list) if series_list else _create_part_raport(df)

def filter_finish(df):
    return df[
        (df.length < to_te)
        & (df.length > from_te)
        & (df.length_genome > -100)
        & (df.length_genome < 12000)
    ]


In [2]:
ont = ["K10B", "K10f", "LK5_1_3"]
ont = ["K10B", "K10f"]
ont = ["konik4"]
genome_path = "genome/genomic2.fna"
te_path = "te/Alex1_3.fas"
dif_percent = 30

te_name = 'Alex1_chr1:18795335-18799058'
te_name = 'Alex1'
te_len = get_len_fasta(te_path)[te_name]
from_te = get_from_te(te_path, te_name)
to_te = get_to_te(te_path, te_name)


In [3]:
headers = ['qseqid', 'sseqid',  'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'expect value', 'bitscore']
headers_dict = {i[0]:i[1] for i in enumerate(headers)}

In [ ]:
import random
def rand_split(sequence, splits):
    for splitLen in splits:
        if splitLen > len(sequence):
            break
        yield sequence[:splitLen]
        sequence = sequence[splitLen::]

def rand_gen(int_list):
    while True:
        yield int(random.choice(int_list))

def cut_genome(genome_path, ont_path, coverage, filename):
    possibility_len = list(get_len_fasta(ont_path).values())
    n = 0
    for i in range(coverage):
        print(i)
        for name, genome_sequence in get_fasta(genome_path):
            print(name)
            with open(filename, 'a') as file:
                for seq in rand_split(genome_sequence, rand_gen(possibility_len)):
                    n = n + 1
                    file.write(f">{name}_{i}_{n}\n{seq}\n")
cut_genome(genome_path, "ont/K10f.fasta", 20, 'cutted_genome.fasta')

0
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
1
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
2
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
3
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
4
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
5
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
6
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
7
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
8
chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9
9
chromoso

In [35]:
import random
w = 2
okno = list(okno)
n = 0
coverage = 1
for i in range(coverage):
    for name, genome_sequence in get_fasta(genome_path):
        print(name)
        with open(f'migi{w}.mi', 'a') as file:
            for seq in rand_split(genome_sequence, rand_gen([30000])):
                n = n + 1
                file.write(f">{name}_{w}_{n}\n")
                file.write(seq)
                file.write("\n")

chromosome1
chromosome2
chromosome3
chromosome4
chromosome5
chromosome6
chromosome7
chromosome8
chromosome9


['123', '456', '789', '0']


In [19]:
def mask_genome(te_name, te_path, genome_path, make_db=False):
    os.system(f"mkdir -p masked_genome")
    if make_db:
        os.system(f"makeblastdb -in {genome_path} -dbtype nucl -out genome/genome")
    os.system(f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db genome/genome -out masked_genome/{te_name}_genome.bl -perc_identity 0.9")
    tex = read_fasta(f"masked_genome/{te_name}_genome.bl")
    tex = tex[tex.qseqid == te_name]
    tex["start"] = tex['sstart'].where(tex['sstart'] <= tex["send"], other=tex['send'])
    tex["stop"] = tex['sstart'].where(tex['sstart'] > tex["send"], other=tex['send'])
    # todo
    save_fasta(tex[["sseqid", "start", "stop"]], f"masked_genome/to_mask_genome_{te_name}.txt")
    plus_tex = tex[tex.sstart < tex.send]
    minus_tex = tex[tex.sstart > tex.send]

    merge_headers = ["sseqid", "sstart", "send"]
    save_fasta(plus_tex[merge_headers].sort_values(by=merge_headers), f"masked_genome/plus_to_mask_genome_{te_name}.bed")
    merge_headers = ["sseqid", "send", "sstart"]
    save_fasta(minus_tex[merge_headers].sort_values(by=merge_headers), f"masked_genome/minus_to_mask_genome_{te_name}.bed")

    os.system(f"bedtools merge -i masked_genome/plus_to_mask_genome_{te_name}.bed -d 200 > masked_genome/plus_to_mask_genome_{te_name}.bed.merged")
    os.system(f"bedtools merge -i masked_genome/minus_to_mask_genome_{te_name}.bed -d 200 > masked_genome/minus_to_mask_genome_{te_name}.bed.merged")
    minus_merged =  pd.read_csv(f"masked_genome/minus_to_mask_genome_{te_name}.bed.merged", sep="\t", header=None)
    plus_merged =  pd.read_csv(f"masked_genome/plus_to_mask_genome_{te_name}.bed.merged", sep="\t", header=None)
    merged_genome = pd.concat(
        [
            minus_merged[
                (minus_merged[2] - minus_merged[1] > from_te)
                & (minus_merged[2] - minus_merged[1] < to_te)
            ],
            plus_merged[
                (plus_merged[2] - plus_merged[1] > from_te)
                & (plus_merged[2] - plus_merged[1] < to_te)
            ],
        ]
    )
    merged_genome["length"] = merged_genome[2] - merged_genome[1]
    save_fasta(merged_genome, f"masked_genome/to_mask_genome_{te_name}.bed.merged")
    os.system(f"bedtools maskfasta -fi {genome_path} -bed masked_genome/to_mask_genome_{te_name}.bed.merged -fo genome/{te_name}_masked_genome")
    return merged_genome
doxi = mask_genome(te_name, te_path, genome_path, True)



Building a new DB, current time: 04/17/2024 05:14:20
New DB name:   /home/kiki/data2/genome/genome
New DB title:  genome/genomic2.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data2/genome/genome
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 5.50287 seconds.




In [41]:
doxi.head(20)

,0,1,2,length
1,chromosome2,30539632,30544357,4725
4,chromosome3,27717880,27722606,4726
6,chromosome5,8967214,8971939,4725
9,chromosome9,9220466,9226148,5682
10,chromosome9,28529339,28534568,5229
0,chromosome1,18795334,18799058,3724
2,chromosome3,41159456,41164182,4726
5,chromosome4,35827649,35832374,4725


In [117]:

oxi.head(30)

,qseqid,sseqid,start_te,end_te,start_genome_te,end_genome_te,qstart1,qend1,sstart1,send1,qstart2,qend2,sstart2,send2,length,length_genome
0,chromosome1_2_627:11335-23058,chromosome1,4000.0,7725.0,18795334.0,18799059.0,7725.0,11724.0,18799059.0,18803058.0,1.0,4000.0,18791335.0,18795334.0,3725.0,3725.0
0,chromosome2_1_3280:10633-23357,chromosome2,4000.0,8726.0,30539632.0,30544358.0,8726.0,12725.0,30544358.0,30548357.0,1.0,4000.0,30535633.0,30539632.0,4726.0,4726.0
0,chromosome3_1_4923:13881-25000,chromosome3,4000.0,8727.0,27717880.0,27722607.0,8727.0,11120.0,27722607.0,27725000.0,1.0,4000.0,27713881.0,27717880.0,4727.0,4727.0
0,chromosome3_1_5461:5457-18182,chromosome3,4000.0,8727.0,41159456.0,41164183.0,8727.0,12726.0,41164183.0,41168182.0,1.0,4000.0,41155457.0,41159456.0,4727.0,4727.0
0,chromosome4_0_9075:3650-16374,chromosome4,4000.0,8726.0,35827649.0,35832375.0,8726.0,12725.0,35832375.0,35836374.0,1.0,4000.0,35823650.0,35827649.0,4726.0,4726.0
0,chromosome4_1_7260:1-11374,chromosome4,2649.0,7375.0,35827649.0,35832375.0,7375.0,11374.0,35832375.0,35836374.0,1.0,2649.0,35825001.0,35827649.0,4726.0,4726.0
0,chromosome4_2_6050:3650-16374,chromosome4,4000.0,8726.0,35827649.0,35832375.0,8726.0,12725.0,35832375.0,35836374.0,1.0,4000.0,35823650.0,35827649.0,4726.0,4726.0
1,chromosome5_0_9528:3215-15939,chromosome5,4000.0,8726.0,8967214.0,8971940.0,8726.0,12725.0,8971940.0,8975939.0,2094.0,4000.0,8965308.0,8967214.0,4726.0,4726.0
1,chromosome5_1_7621:13215-25000,chromosome5,4000.0,8726.0,8967214.0,8971940.0,8726.0,11786.0,8971940.0,8975000.0,2094.0,4000.0,8965308.0,8967214.0,4726.0,4726.0
0,chromosome9_2_11247:6467-20148,chromosome9,4000.0,9683.0,9220466.0,9226149.0,9683.0,13682.0,9226149.0,9230148.0,1.0,4000.0,9216467.0,9220466.0,5683.0,5683.0


In [36]:
def first_part():
    headers = ['qseqid', 'sseqid',  'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'expect value', 'bitscore']
    headers_dict = {i[0]:i[1] for i in enumerate(headers)}
    os.system("mkdir -p blast")
    os.system("mkdir -p first")
    if True:
        #for file in ont:
        #    os.system(f"seqtk seq -a ont/{file}.fastq.gz > ont/{file}.fasta")
        #!seqtk seq -a ont/K10B.fastq.gz > ont/K10B.fasta
        for file in ont:
            os.system(f"makeblastdb -in ont/{file}.fasta -dbtype nucl -out ont/{file}")
        #!makeblastdb -in ont/K10B.fasta -dbtype nucl -out ont/K10B
        for file in ont:
            os.system(f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db ont/{file} -out blast/TE_{file}.bl -dust no -perc_identity 0.9")
        #!blastn -num_threads 20 -outfmt 6 -query te/Alex1_Alex3.fasta -db ont/K10B -out blast/Alex_K10B.bl -dust no -perc_identity 0.9

    for o in ont:
        te_ont_blast = read_fasta(f"blast/TE_{o}.bl")
    
        alex1 = te_ont_blast[te_ont_blast.qseqid.str.contains('Alex1')]
        plus_alex1 = alex1[alex1.sstart < alex1.send]
        minus_alex1 = alex1[alex1.sstart > alex1.send]
    
    
        p_bed_df = plus_alex1[['sseqid', 'sstart', 'send', 'qseqid']].sort_values(by=['sseqid', 'sstart', 'send'])
        m_bed_df = minus_alex1[['sseqid', 'send', 'sstart', 'qseqid']].sort_values(by=['sseqid', 'send', 'sstart'])
    
        save_fasta(m_bed_df, f'first/m_raw_first_blast_{o}.bed')
        save_fasta(p_bed_df, f'first/p_raw_first_blast_{o}.bed')

        os.system(f"bedtools merge -i first/p_raw_first_blast_{o}.bed -d 1500 -c 4 -o collapse  > first/p_first_blast_{o}.bed.merged")
        os.system(f"bedtools merge -i first/m_raw_first_blast_{o}.bed -d 1500 -c 4 -o collapse  > first/m_first_blast_{o}.bed.merged")

        print("first part")
    
        p_merged = pd.read_csv(
            f"first/p_first_blast_{o}.bed.merged", sep="\t", header=None).rename(
            {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
        )
        m_merged = pd.read_csv(
            f"first/m_first_blast_{o}.bed.merged", sep="\t", header=None).rename(
            {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
        )
    
        p_filtered_first_bed = p_merged[(p_merged.send - p_merged.sstart) > from_te]
        m_filtered_first_bed = m_merged[(m_merged.send - m_merged.sstart) > from_te]
    
        pd.options.mode.copy_on_write = True
        p_filtered_first_bed['end'] = p_filtered_first_bed['send'] + 4000
        p_filtered_first_bed['start'] = p_filtered_first_bed['sstart'] - 4000
        save_fasta(p_filtered_first_bed[['sseqid', 'sstart', 'send']], f'first/p_normal_{o}.txt')
    
        m_filtered_first_bed['end'] = m_filtered_first_bed['send'] + 4000
        m_filtered_first_bed['start'] = m_filtered_first_bed['sstart'] - 4000
        save_fasta(m_filtered_first_bed[['sseqid', 'sstart', 'send']], f'first/m_normal_{o}.txt')
    
        save_fasta(p_filtered_first_bed[['sseqid']], f'first/p_seq_list_{o}.txt')
        save_fasta(m_filtered_first_bed[['sseqid']], f'first/m_seq_list_{o}.txt')
        print("second part")
    
        #long
        os.system(f"seqtk subseq ont/{o}.fasta first/p_seq_list_{o}.txt > first/p_normal_{o}.fasta")
        os.system(f"seqtk subseq ont/{o}.fasta first/m_seq_list_{o}.txt > first/m_normal_{o}.fasta")
        print("1 part")

        os.system(f"bedtools maskfasta -fi first/p_normal_{o}.fasta -bed first/p_normal_{o}.txt -fo first/p_Alex_{o}.masked.fasta")
        os.system(f"seqkit fx2tab --length --name  first/p_Alex_{o}.masked.fasta > first/p_max_lenght_{o}.txt")
        print("2 part")
        p_max_len = pd.read_csv(
            f'first/p_max_lenght_{o}.txt', sep="\t", header=None).rename(
            {0:'sseqid', 1:'max_len'}, axis=1).sort_values(by=['sseqid'])
        #print("3 part")
        os.system(f"bedtools maskfasta -fi first/m_normal_{o}.fasta -bed first/m_normal_{o}.txt -fo first/m_Alex_{o}.masked.fasta")
        os.system(f"seqkit fx2tab --length --name  first/m_Alex_{o}.masked.fasta > first/m_max_lenght_{o}.txt")
        print("4 part")
    
        m_max_len = pd.read_csv(
            f'first/m_max_lenght_{o}.txt', sep="\t", header=None).rename(
            {0:'sseqid', 1:'max_len'}, axis=1).sort_values(by=['sseqid'])
    
        #sseqid bo query to po prostu te w tym przypadku
        save_fasta(p_filtered_first_bed[['sseqid', 'start', 'end']], 'p_cutted_{o}.txt')
        save_fasta(m_filtered_first_bed[['sseqid', 'start', 'end']], 'm_cutted_{o}.txt')
        print("third part")

        """ """
        p_mask_min = p_filtered_first_bed.start < 0
        p_filtered_first_bed.loc[p_mask_min, 'start'] = 0
        p_filtered_first_bed = pd.merge(p_filtered_first_bed, p_max_len, how='left', on="sseqid")
        p_mask_max = p_filtered_first_bed.end > p_filtered_first_bed.max_len
        p_filtered_first_bed.loc[p_mask_max, 'end'] = p_filtered_first_bed.max_len
        save_fasta(p_filtered_first_bed[['sseqid', 'start', 'end']], f'first/p_cutted2_{o}.txt')
        print("forth part")
        """ """
    
        m_mask_min = m_filtered_first_bed.start < 0
        m_filtered_first_bed.loc[m_mask_min, 'start'] = 0
        m_filtered_first_bed = pd.merge(m_filtered_first_bed, m_max_len, how='left', on="sseqid")
        m_mask_max = m_filtered_first_bed.end > m_filtered_first_bed.max_len
        m_filtered_first_bed.loc[m_mask_max, 'end'] = m_filtered_first_bed.max_len
        save_fasta(m_filtered_first_bed[['sseqid', 'start', 'end']], f'first/m_cutted2_{o}.txt')
    
        # bedtools getfasta
        os.system(f"seqtk subseq first/p_Alex_{o}.masked.fasta first/p_cutted2_{o}.txt > first/p_cutted_ont_{o}.fasta")
        os.system(f"seqtk subseq first/m_Alex_{o}.masked.fasta first/m_cutted2_{o}.txt > first/m_cutted_ont_{o}.fasta")
        print("fifth part")


In [37]:
first_part()



Building a new DB, current time: 04/17/2024 05:44:24
New DB name:   /home/kiki/data2/ont/konik4
New DB title:  ont/konik4.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 44629 sequences in 13.0739 seconds.




first part
second part
1 part
2 part
4 part
third part
forth part
fifth part


In [102]:
h = [0, "from", "to"]
save_fasta(doxi[h], 'doxi')
!seqtk subseq genome/masked_DCARv3.4.fa doxi > dixi.fasta

In [103]:
!mkdir okno
os.system("makeblastdb -in ont/K10B.fasta -dbtype nucl -out okno/okno")




Building a new DB, current time: 04/08/2024 05:52:47
New DB name:   /home/kiki/data/okno/okno
New DB title:  ont/K10B.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6939343 sequences in 341.898 seconds.




0

In [104]:
os.system("blastn -num_threads 20 -outfmt 6 -query dixi.fasta -db okno/okno -out wuli -perc_identity 0.9")


0

# Second part of analysis

In [114]:
def second_part(blast_db=False):
    all_report = []
    os.system("mkdir -p final")
    os.system("mkdir -p second")
    pd.options.mode.copy_on_write = True
    if blast_db:
        # todo
        os.system(f"makeblastdb -in {genome_path}  -dbtype nucl -out genome/masked_genome")
    for o in ont:
        #os.system("blastn -num_threads 20 -outfmt 6 -query p_cutted_ont.fasta -db genome/masked_DCARv3.4 -out p_Alex_K10B_genome.bl -perc_identity 0.8 -max_target_seqs 4")
        os.system(f"blastn -num_threads 20 -outfmt 6 -query first/p_cutted_ont_{o}.fasta -db genome/masked_genome -out second/p_Alex_{o}_genome.bl -perc_identity 0.9")
        #os.system("blastn -num_threads 20 -outfmt 6 -query m_cutted_ont.fasta -db genome/masked_DCARv3.4 -out m_Alex_K10B_genome.bl -perc_identity 0.8 -max_target_seqs 4")
        os.system(f"blastn -num_threads 20 -outfmt 6 -query first/m_cutted_ont_{o}.fasta -db genome/masked_genome -out second/m_Alex_{o}_genome.bl -perc_identity 0.9")
    
        plus_blast = pd.read_csv(f"second/p_Alex_{o}_genome.bl", sep="\t", header=None).rename(headers_dict, axis=1)
        minus_blast = pd.read_csv(f"second/m_Alex_{o}_genome.bl", sep="\t", header=None).rename(headers_dict, axis=1)
    
        p_plus_blast = plus_blast[plus_blast.sstart <= plus_blast.send]
        m_plus_blast = plus_blast[plus_blast.sstart > plus_blast.send]
        
        p_minus_blast = minus_blast[minus_blast.sstart <= minus_blast.send]
        m_minus_blast = minus_blast[minus_blast.sstart > minus_blast.send]
    
        p_plus = create_report(p_plus_blast)
        m_plus = create_report(m_plus_blast)
        p_minus = create_report(p_minus_blast)
        m_minus = create_report(m_minus_blast)

        #todo beter filter
        p_plus = create_report(p_plus_blast[(p_plus_blast.length > 1500)])
        m_plus = create_report(m_plus_blast[(m_plus_blast.length > 1500)])
        p_minus = create_report(p_minus_blast[(p_minus_blast.length > 1500)])
        m_minus = create_report(m_minus_blast[(m_minus_blast.length > 1500)])
        
        #p_plus.to_csv(f"second/p_plus_report_{o}", sep='\t', index=False, encoding='utf-8', header=True) # i add header
        #m_plus.to_csv(f"second/m_plus_report_{o}", sep='\t', index=False, encoding='utf-8', header=True) # i add header
        #p_minus.to_csv(f"second/p_minus_report_{o}", sep='\t', index=False, encoding='utf-8', header=True) # i add header
        #m_minus.to_csv(f"second/m_minus_report_{o}", sep='\t', index=False, encoding='utf-8', header=True) # i add header
    
        p_plus_final = filter_finish(p_plus)
        m_plus_final = filter_finish(m_plus)
        p_minus_final = filter_finish(p_minus)
        m_minus_final = filter_finish(m_minus)
        
        os.system("mkdir -p final")
        save_fasta(p_plus_final, f'final/p_plus_final_{o}')
        save_fasta(m_plus_final, f'final/m_plus_final_{o}')
        save_fasta(p_minus_final, f'final/p_minus_final_{o}')
        save_fasta(m_minus_final, f'final/m_minus_final_{o}')
        
        final_concat =  pd.concat(
            [
                p_plus_final, 
                m_plus_final, 
                p_minus_final, 
                m_minus_final, 
            ]
        ).sort_values(by=['sseqid', 'start_genome_te'])
        all_report.append(final_concat)
    
        #fin[final_.end_genome_te - ni.start_genome_te > 20]
        #return final_concat
    return pd.concat(all_report)
    
oxi = second_part(True)



Building a new DB, current time: 04/17/2024 22:23:31
New DB name:   /home/kiki/data2/genome/masked_genome
New DB title:  genome/genomic2.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data2/genome/masked_genome
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 4.86382 seconds.




In [44]:
oxi

,qseqid,sseqid,start_te,end_te,start_genome_te,end_genome_te,qstart1,qend1,sstart1,send1,qstart2,qend2,sstart2,send2,length,length_genome
1,chromosome5_0_9528:3215-15939,chromosome5,4000,8726,8967214,8971940,8726,12725,8971940,8975939,2094,4000,8965308,8967214,4726,4726
1,chromosome5_1_7621:13215-25000,chromosome5,4000,8726,8967214,8971940,8726,11786,8971940,8975000,2094,4000,8965308,8967214,4726,4726
0,chromosome9_0_16872:1-10148,chromosome9,466,6149,9220466,9226149,6149,10148,9226149,9230148,1,466,9220001,9220466,5683,5683
7,chromosome9_0_17837:5340-18568,chromosome9,3999,9230,28345736,28345740,9230,9437,28345740,28345533,3923,3999,28345810,28345736,5231,4
7,chromosome9_1_14269:340-13568,chromosome9,3999,9230,28345736,28345740,9230,9437,28345740,28345533,3923,3999,28345810,28345736,5231,4


In [118]:
def mid(rec, row):
    start =  row['start_genome_te']
    for m in rec.keys():
        if start - m < 1000 and start - m > -1000:
            rec[m][2] =  rec[m][2] + 1
            return
    rec[start] =  [row['start_genome_te'], row['end_genome_te'], 1]
    return
    

records = {}
for sid in oxi.sseqid.unique():
    okno = oxi[(oxi.sseqid == sid)]
    records[sid] = {}
    for index, row in okno.iterrows():
        mid(records[sid], row) 


records

{'chromosome1': {18795334.0: [18795334.0, 18799059.0, 1]},
 'chromosome2': {30539632.0: [30539632.0, 30544358.0, 1]},
 'chromosome3': {27717880.0: [27717880.0, 27722607.0, 1],
  41159456.0: [41159456.0, 41164183.0, 1]},
 'chromosome4': {35827649.0: [35827649.0, 35832375.0, 3]},
 'chromosome5': {8967214.0: [8967214.0, 8971940.0, 2]},
 'chromosome9': {9220466.0: [9220466.0, 9226149.0, 1],
  28529339.0: [28529339.0, 28534569.0, 2]}}

In [89]:
oxi.head()

,qseqid,sseqid,start_te,end_te,start_genome_te,end_genome_te,qstart1,qend1,sstart1,send1,qstart2,qend2,sstart2,send2,length,length_genome
0,9676ad26-b838-4bf4-b6f3-803f3cb06ff0:8506-20811,chromosome1,4000,8307,1895047,1895034,8307,9048,1895034,1894286,1,4000,1899099,1895047,4307,-13
0,2c40afb0-7c90-4ddf-847c-a587e291d4a9:1-9021,chromosome1,612,5042,1895047,1895016,5042,9021,1895016,1890954,31,612,1895639,1895047,4430,-31
9,08051692-8a2e-4ada-ae64-5ded5fdb190a:28450-38629,chromosome1,3821,8404,12813067,12813069,8404,9604,12813069,12814278,2697,3821,12811946,12813067,4583,2
0,9ebca745-06aa-4187-9303-3fc73853bf74:1-10655,chromosome1,2054,6658,27115104,27115102,6658,10655,27115102,27111051,27,2054,27117184,27115104,4604,-2
2,98c0c670-f831-449a-a2d7-6b50aad7d341:308-12808,chromosome1,4000,8546,40590593,40590596,8546,10780,40590596,40588308,1173,4000,40593451,40590593,4546,3
